# Purpose

Raw Scopus files to UK affiliations + their data from Scopus.

Upload.

# Import dependencies

In [258]:
import pandas as pd
import numpy as np
import glob
import os
from pybliometrics.scopus import AffiliationRetrieval
import requests
import json

# Cleaning data

In [2]:
def get_csv_files(test_run=True):
    if test_run:
        return ['bristol_test_run.csv']
    else:
        os.chdir(os.getcwd() + '\\scopus')
        return glob.glob('*.csv')


def find_indices(df):
    
    def get_country_indices(x, country='United Kingdom'):
        return [1 if i == country else 0 for i in x]
    
    df['encoding'] = df['affiliation_country'].apply(get_country_indices)
    
    return df
 
    
def get_affiliations(csv_files):
    
    dfs = []
    
    for file in csv_files:
        
        print(file)
        df = pd.read_csv(file, usecols=['afid', 'affilname', 'affiliation_city', 'affiliation_country'])
        
        # Split affiliation and country into lists
        df['original_affilname'] = df['affilname'].copy()
        
        # Remove special character anomolies
        df['affilname'] = df.affilname.str.replace('&amp;', '')
        df['affilname'] = df.affilname.str.replace('acute;', '')
        
        # Split the fields by semi-colon
        df['afid'] = df['afid'].str.split(pat=';')
        df['affilname'] = df['affilname'].str.split(pat=';')
        df['affiliation_city'] = df['affiliation_city'].str.split(pat=';')
        df['affiliation_country'] = df['affiliation_country'].str.split(pat=';')
        
        # Create empty list if nan - deal with bug finding indices
        bug_catcher = lambda x: x if isinstance(x, list) else []
        df['affilname'] = df['affilname'].apply(bug_catcher)
        df['affiliation_country'] = df['affiliation_country'].apply(bug_catcher)
        
        # Retrieve only UK institutions
        df = find_indices(df)
        
        dfs.append(df)
    
    return dfs

In [3]:
def add_uk_affils_only(df):
    
    # Add columns for UK only to df
    df['uk_afid'] = None
    df['uk_affil_name'] = None
    df['uk_affil_city'] = None
    
    id_errors = 0
    name_errors = 0
    city_errors = 0
    
    # Loop through df and create an array of only uk affiliations
    for idx, row in df.iterrows():
        
        # Create boolean array
        bool_list = list(map(bool, row['encoding']))
        encoding_array = np.array(bool_list) 
        
        # Create arrays to be indexed by boolean array
        affil_array = np.array(row['affilname'])
        afid_array = np.array(row['afid'])
        affil_city_array = np.array(row['affiliation_city'])
        
        # Try setting UK affiliation id array
        try:
            uk_afid = afid_array[encoding_array]
            df.iloc[idx, 6] = uk_afid
        except:
            #print('#################################')
            #print('Problem setting id array:', idx)
            df.iloc[idx, 6] = np.nan
            #print(row)
            id_errors += 1
        
        # Try setting UK affiliation name array
        try:
            uk_affil_name = affil_array[encoding_array]
            df.iloc[idx, 7] = uk_affil_name
        except:
            #print('#################################')
            #print('Problem setting affiliation name array:', idx)
            df.iloc[idx, 7] = np.nan
            #print(row)
            name_errors += 1
        
        # Try setting UK affiliation city array
        try:
            uk_affil_city = affil_city_array[encoding_array]
            df.iloc[idx, 8] = uk_affil_city
        except:
            #print('#################################')
            #print('Problem setting affiliation city array:', idx)
            df.iloc[idx, 8] = np.nan
            #print(row)
            city_errors += 1
    
    print('ID errors:', id_errors)
    print('Name errors:', name_errors)
    print('City errors:', city_errors)

    return df

# Execute

If there is an error with city, put the original city array in (even if there are non-UK countries).

In [4]:
csv_files = get_csv_files(test_run=False)
print('csv files in cd:\n', csv_files)

print('Reading csv files and finding UK indices...')
dfs = get_affiliations(csv_files)

csv files in cd:
 ['2000.csv', '2001.csv', '2002.csv', '2003.csv', '2004.csv', '2005.csv', '2006.csv', '2007.csv', '2008.csv', '2009.csv']
Reading csv files and finding UK indices...
2000.csv
2001.csv
2002.csv
2003.csv
2004.csv
2005.csv
2006.csv
2007.csv
2008.csv
2009.csv


In [5]:
for i, df in enumerate(dfs):
    
    print(f'Adding UK affils for {csv_files[i]}...')
    
    # Remove papers that do not have an affiliation
    num_papers = df.shape[0]
    df = df.dropna(subset=['afid'])
    papers_removed = num_papers - df.shape[0]
    print(f'Removed {papers_removed} of {num_papers} papers due to no affiliation given.')
    
    # Reset index
    df = df.reset_index(drop=True)
    
    # Format UK only affiliations
    dfs[i] = add_uk_affils_only(df)

Adding UK affils for 2000.csv...
Removed 1032 of 80202 papers due to no affiliation given.
ID errors: 28
Name errors: 33
City errors: 535
Adding UK affils for 2001.csv...
Removed 1258 of 75136 papers due to no affiliation given.
ID errors: 13
Name errors: 19
City errors: 507
Adding UK affils for 2002.csv...
Removed 1136 of 72918 papers due to no affiliation given.
ID errors: 20
Name errors: 32
City errors: 518
Adding UK affils for 2003.csv...
Removed 848 of 72855 papers due to no affiliation given.
ID errors: 16
Name errors: 21
City errors: 337
Adding UK affils for 2004.csv...
Removed 911 of 75884 papers due to no affiliation given.
ID errors: 47
Name errors: 50
City errors: 439
Adding UK affils for 2005.csv...
Removed 762 of 78348 papers due to no affiliation given.
ID errors: 28
Name errors: 32
City errors: 438
Adding UK affils for 2006.csv...
Removed 1063 of 86931 papers due to no affiliation given.
ID errors: 61
Name errors: 64
City errors: 475
Adding UK affils for 2007.csv...
Remo

__Papers that had ID errors__

These were errors because they do not have country data and therefore could not produce an encoding array for country. It seems like the majority of them are just on institution and are UK related (due to the UK search). Therefore, I'm going to keep these in the data set.

In [6]:
# Look at the papers that had ID errors
# These were errors because they do not have country
dfs[0][dfs[0]['uk_afid'].isnull()].shape

(28, 9)

__Papers that had name errors__

These include the same errors as the above BUT there are more. The additional errors come from poor formatting of the institute names. I think it comes from the fact that some institute names have ';' within them. This means that they are split into different institutions during the extraction of data. Since these are relatively few and far between, I don't think it is unreasonable to drop these. For example, 2000 has 5, 2001 has 6, 2002 has 12.

In [8]:
# Look at the papers that had name errors
dfs[0][dfs[0]['uk_affil_name'].isnull()].shape

(33, 9)

__Papers that had city errors__

These cause errors because the affiliation does not have a city associated with it. This could have happened because a non-UK institution didn't have a city with it though. I would like to keep the papers that only have affilname.

I would like to look at the entries that have length more than 1 for affilname - none of them are! Therefore, I am happy to keep them all. To keep these records, the uk_afid needs to be complete.

In [27]:
dfs[0].head()

,afid,affilname,affiliation_city,affiliation_country,original_affilname,encoding,uk_afid,uk_affil_name,uk_affil_city
0,"[60030480, 60022871]","[University of Bath, St Mary's Hospital]","[Bath, London]","[United Kingdom, United Kingdom]",University of Bath;St Mary's Hospital,"[1, 1]","[60030480, 60022871]","[University of Bath, St Mary's Hospital]","[Bath, London]"
1,"[60022148, 60011520]","[University College London, King's College Lon...","[London, London]","[United Kingdom, United Kingdom]",University College London;King's College London,"[1, 1]","[60022148, 60011520]","[University College London, King's College Lon...","[London, London]"
2,"[60030480, 60022871]","[University of Bath, St Mary's Hospital]","[Bath, London]","[United Kingdom, United Kingdom]",University of Bath;St Mary's Hospital,"[1, 1]","[60030480, 60022871]","[University of Bath, St Mary's Hospital]","[Bath, London]"
3,[60003771],[The University of Manchester],[Manchester],[United Kingdom],The University of Manchester,[1],[60003771],[The University of Manchester],[Manchester]
4,[60011520],[King's College London],[London],[United Kingdom],King's College London,[1],[60011520],[King's College London],[London]


In [35]:
# If uk_affil_city is null and affilname is length 1, add affilname to uk_affil_name
# and afid to uk_afid
def add_uk_afid(x):
    if (isinstance(x['uk_affil_city'], float)) & (len(x['affilname']) == 1):
        return x['afid']
    else:
        return x['uk_afid']
    
def add_uk_affilname(x):
    if (isinstance(x['uk_affil_city'], float)) & (len(x['affilname']) == 1):
        return x['affilname']
    else:
        return x['uk_affil_name']


for df in dfs:
    df['uk_afid'] = df.apply(lambda x: add_uk_afid(x), axis=1)
    df['uk_affil_name'] = df.apply(lambda x: add_uk_affilname(x), axis=1)

In [46]:
dfs[0][dfs[0]['uk_afid'].str.len() != dfs[0]['uk_affil_name'].str.len()]

,afid,affilname,affiliation_city,affiliation_country,original_affilname,encoding,uk_afid,uk_affil_name,uk_affil_city
999,"[60022144, 60010396, 60008950, 60001490, 10031...","[University of Central Florida, CSIC - Institu...","[Orlando, Madrid, Canberra, Glasgow, Linthicum]","[United States, Spain, Australia, United Kingd...",University of Central Florida;CSIC - Instituto...,"[0, 0, 0, 1, 0]",[60001490],NaN,[Glasgow]
15087,"[60025225, 60010396]","[University of Southampton, CSIC - Instituto d...","[Southampton, Madrid]","[United Kingdom, Spain]",University of Southampton;CSIC - Instituto de ...,"[1, 0]",[60025225],NaN,[Southampton]
26322,"[60015150, 60010396, 60007643]","[Imperial College London, CSIC - Instituto de ...","[London, Madrid, Madrid]","[United Kingdom, Spain, Spain]",Imperial College London;CSIC - Instituto de Óp...,"[1, 0, 0]",[60015150],NaN,[London]
27929,"[60138535, 60123796, 60029857, 60025225, 60021...",[Laboratoire de Chimie et de Physique Approche...,"[Metz, Paris, Heraklion, Southampton, Moscow, ...","[France, France, Greece, United Kingdom, Russi...",Laboratoire de Chimie et de Physique Approche ...,"[0, 0, 0, 1, 0, 0, 0]",[60025225],NaN,[Southampton]
30269,"[60018940, 60017317, 60010396, 101702760]","[Universidad Rey Juan Carlos, University of Su...","[Mostoles, Brighton, Madrid, Ringmer]","[Spain, United Kingdom, Spain, United Kingdom]",Universidad Rey Juan Carlos;University of Suss...,"[0, 1, 0, 1]","[60017317, 101702760]",NaN,"[Brighton, Ringmer]"


# Format affiliations

This consolidates the UK institutions from different dataframes into one dataframe.

In [71]:
def format_dfs(dfs):
    
    af_ids = []
    affil_names = []
    affil_cities = []
    
    for df in dfs:
        
        for i, row in df.iterrows():
            
            names = row['uk_affil_name']
            # Skip papers where UK num_ids != num_names (due to name splitting error)
            if isinstance(names, float):
                if pd.isnull(names):
                    continue
            # Skip papers with no UK institutions that somehow sneaked in
            elif isinstance(row['encoding'], np.ndarray):
                if names.all() == 0:
                    continue
            
            # Loop through each institution associated with this paper
            for j, afid in enumerate(row['uk_afid']):
                
                # If institution not already in list, add to list
                if afid not in af_ids:
                    af_ids.append(afid)
                    affil_names.append(row['uk_affil_name'][j])
                    # Some affiliations do not have city data
                    try:
                        affil_cities.append(row['uk_affil_city'][j])
                    except:
                        affil_cities.append(np.nan)
    
    d = {'af_id': af_ids, 'affil_name': affil_names, 'affil_city': affil_cities}
    formatted_df = pd.DataFrame(data=d)
    
    return formatted_df


scopus_uk_affils_df = format_dfs(dfs)

# Quick look at the data

* 48060 affiliations
* 5068 affiliation IDs have '' for city
* 3659 affiliation IDs have NaN for city
* There are some non-UK institutions in the data

In [140]:
scopus_uk_affils_df[scopus_uk_affils_df['affil_city'].isnull()]

,af_id,affil_name,affil_city
108,100807854,Orthopaedic SpR,NaN
315,100519274,D.E.R.A.,NaN
573,100882603,"ESAB UK, Ltd",NaN
574,100915017,Armology (UK) Ltd,NaN
589,101702583,National Phjsical Laboratory,NaN
...,...,...,...
47906,109869100,Anesteziologicko-resuscitační Klinika,NaN
47913,109673652,Glamorgan Blended Learning Ltd,NaN
47921,107146571,University School of Dentistry,NaN
47927,116423534,MARITIME CARGO PROCESSING PLC,NaN


5219 7935 10883 17525 39466 are Chicago! Checking this against the original data, it seems as though this is an error by Scopus (or whoever provided the data there).

In [108]:
# How has Chicago ended up in there?
for i, df in enumerate(dfs):
    for j, row in df.iterrows():
        cities = row['uk_affil_city']
        if isinstance(cities, np.ndarray):
            if 'Chicago' in cities:
                print(i)
                print(row)
                break
        else:
            continue
    break

0
afid                           [110526565]
affilname              [DePaul University]
affiliation_city                 [Chicago]
affiliation_country       [United Kingdom]
original_affilname       DePaul University
encoding                               [1]
uk_afid                        [110526565]
uk_affil_name          [DePaul University]
uk_affil_city                    [Chicago]
Name: 39284, dtype: object


In [132]:
temp = pd.read_csv('2000.csv')

In [133]:
temp['help'] = temp['affiliation_city'].str.split(pat=';')
temp[(temp.help.str.len() == 1) & (temp.affiliation_city.str.contains('Chicago'))]

,Unnamed: 0,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,affiliation_city,affiliation_country,author_count,author_names,author_ids,author_afids,coverDate,coverDisplayDate,publicationName,issn,source_id,eIssn,aggregationType,volume,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor,help
39790,39790,2-s2.0-85013772732,10.1017/S0010417500002607,NaN,NaN,Society Reified: Herbert Spencer and Political...,ar,Article,Howland D.,110526565,DePaul University,Chicago,United Kingdom,1.0,"Howland, Douglas",24343938900,110526565,2000-01-01,2000,Comparative Studies in Society and History,00104175,17670,14752999,Journal,42,1,NaN,67-86,NaN,NaN,16,0,NaN,undefined,NaN,[Chicago]
50105,50105,2-s2.0-0033825510,10.1080/13651500050518091,NaN,NaN,The efficacy of zotepine in treating acute neg...,ar,Article,Wighton A.,60001681,Abbott Laboratories,Chicago,United States,6.0,"Wighton, A.;Tweed, J. A.;Butler, A.;Welch, C. ...",6506040894;57207608984;57197142435;7202750995;...,60001681;60001681;60001681;60001681;60001681;6...,2000-01-01,2000,International Journal of Psychiatry in Clinica...,13651501,16180,NaN,Journal,4,3,NaN,209-214,INTRODUCTION: Zotepine is a unique antipsychot...,Meta-analysis | Negative symptoms | SANS | Sch...,4,0,NaN,undefined,NaN,[Chicago]


# Obtain affiliation type and address through Affiliation Retrieval API

In [141]:
aff = AffiliationRetrieval('60020650')
print(aff)

University of Bristol in Bristol in United Kingdom,
has 21,832 associated author(s) and 149,442 associated document(s) as of 2021-06-09


In [142]:
aff.postal_code

'BS8 1QU'

In [264]:
def get_response(affiliation_id, key):
    url = f'https://api.elsevier.com/content/affiliation/affiliation_id/{affiliation_id}'
    return requests.get(url, headers = {'Accept': 'application/json', 'X-ELS-APIKey': key})


def get_data(resp):
    
    try:
        post_code = resp.json()['affiliation-retrieval-response']['institution-profile']['address']['postal-code']
    except:
        post_code = np.nan

    try:
        address = resp.json()['affiliation-retrieval-response']['institution-profile']['address']['address-part']
    except:
        address = np.nan

    try:
        affil_type = resp.json()['affiliation-retrieval-response']['institution-profile']['org-type']
    except:
        affil_type = np.nan
    
    return (post_code, address, affil_type)


def use_api(affiliation_id):
    
    keys = ['241a224e08e5a32cc6d3b71c72df0d0f', '1de0de6ccdd919164874776b86a63618',
           '2e746b287d5acef3ca869c8960311102', '5ee68844419ab2204ea7f04dae6626a0',
           '52b9cc612f710a22fc2c81b2ba695d5b', '2e719850293712ecf53f04bed29edb06']
    
    error_codes = [200, 404]  # good request, resource not found
    
    for key in keys:
        resp = get_response(affiliation_id, key)
        if resp.status_code in error_codes:
            break
    
    (post_code, address, affil_type) = get_data(resp)
    
    return (post_code, address, affil_type)


# Testing
#(post_code, address, affil_type) = use_api(107918540)

In [267]:
def get_affil_data(df):
    
    for i, row in df.iterrows():
        
        not_working = [4423, 5413, 9093, 10024, 16013, 18427]
        if i in not_working:
            continue
        if row['post_code'] == '':
            #aff = AffiliationRetrieval('{}'.format(row['af_id']))
            #df.iloc[i, 3] = aff.address
            #df.iloc[i, 4] = aff.postal_code
            #df.iloc[i, 5] = aff.org_type
            try:
                (post_code, address, affil_type) = use_api('{}'.format(row['af_id']))
                df.iloc[i, 3] = address
                df.iloc[i, 4] = post_code
                df.iloc[i, 5] = affil_type
            except Exception as e:
                print(e)
                return df
        else:
            continue
            
    return df

In [199]:
full_scopus_affil_df = scopus_uk_affils_df.copy()
full_scopus_affil_df['affil_address'] = ''
full_scopus_affil_df['post_code'] = ''
full_scopus_affil_df['affil_type'] = ''

In [268]:
# JUST UNDER 24 HOURS PROCESSING TIME
full_scopus_affil_df = get_affil_data(full_scopus_affil_df)

__Summary of data from Scopus__

Of the 48060 affiliations:

* 16537 do not have post code
* 45132 do not have affil type
* 16422 do not have post code or affil type

Of the 2928 that have affil type:

* 887 are companies
* 134 are government
* 807 are hospitals
* 370 are universities
* 297 are residencies
* 238 are national governments

In [284]:
full_scopus_affil_df.sample(n=20)

,af_id,affil_name,affil_city,affil_address,post_code,affil_type
38268,101178414,Chartered Institute of Arbitrators,London,39 Essex Chambers,EC1V 2RS,NaN
17877,60099910,STMicroelectronics Ltd - Bristol,Bristol,1000 Aztec West,BS324SQ,comp
3398,101550621,Survey-2-CAD Ltd,NaN,None,None,None
45893,107322709,People's Open Access Education Initiative (Peo...,Manchester,Eccles,M30 9HN,NaN
44487,101654281,Pain Management Unit,Bath,NaN,NaN,NaN
33190,100822160,Public Administration and Development,London,7 High Street,W5 5DB,NaN
20483,118820470,Care in the Community Function,Newcastle,126 Great Lime Road,NE12 7DQ,NaN
22366,101940669,360 is us Ltd.,Broadstone,NaN,NaN,NaN
4588,101629387,Asmara Resources,London,None,None,None
47416,119859082,Race on the Agenda (ROTA),NaN,NaN,NaN,NaN


# Save csv

In [286]:
os.chdir('..')
full_scopus_affil_df.to_csv('scopus_uk_affils.csv')